# 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
print(df.dtypes)
print(df.isna().sum())
df.head(5)

Name            object
Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales        object
EU_Sales        object
JP_Sales        object
Other_Sales     object
dtype: object
Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [2]:
def unitConversion(v): #기본 단위를 M으로 통일
  if 'K' in v:
    return float(v.replace('K','')) * 0.001

  return float(v.replace('M', ''))

sale_columns = df.filter(regex='_Sales').columns  #_Sales 포함한 컬럼 선택
for c in sale_columns:
  df[c] = df[c].apply(unitConversion)

df['Total_Sales'] = df[sale_columns].sum(axis=1)
df.dtypes

Name            object
Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales       float64
EU_Sales       float64
JP_Sales       float64
Other_Sales    float64
Total_Sales    float64
dtype: object

In [3]:
since1900 = (df['Year'] < 100) & (df['Year'] >= 30)
df[since1900].head(5)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
81,Treasures of the Deep,PS,97.0,Action,Namco Bandai Games,0.10,0.07,0.00,0.01,0.18
906,Mini-Yonku Shining Scorpion: Let's & Go!!,SNES,96.0,Racing,ASCII Entertainment,0.00,0.00,0.90,0.00,0.90
2429,Casper,PS,96.0,Adventure,Interplay,0.46,0.31,0.00,0.05,0.82
3492,Glover,N64,98.0,Platform,Hasbro Interactive,0.72,0.17,0.00,0.01,0.90
4365,Adventure Island,NES,86.0,Platform,Hudson Soft,0.00,0.00,1.05,0.00,1.05


In [4]:
since2000 = (df['Year'] >= 0) & (df['Year'] < 30)
df[since2000].head(5)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
74,Age of Empires III: Complete Collection,PC,9.0,Strategy,Microsoft Game Studios,0.01,0.12,0.0,0.03,0.16
213,NASCAR Unleashed,PS3,11.0,Racing,Activision,0.09,0.00,0.0,0.01,0.10
304,Kung Fu Panda: Showdown of Legendary Legends,PS3,15.0,Action,Little Orbit,0.00,0.02,0.0,0.00,0.02
768,NASCAR Thunder 2004,PS2,3.0,Racing,Electronic Arts,0.53,0.41,0.0,0.14,1.08
1015,Wolfenstein,PC,9.0,Shooter,Activision,0.01,0.03,0.0,0.01,0.05


In [5]:
def cleanYear(year):
  if year < 30:
    return year + 2000
  elif year < 100:
    return year + 1900
  else:
    return year

df['Year'] = df['Year'].apply(cleanYear)
df['Year'].fillna(df['Year'].mean(), inplace = True)  #결측치 평균값으로 처리
df['Year'] = df['Year'].apply(int)  #연도 int로 변경

since1900 = (df['Year'] < 100) & (df['Year'] >= 30)
since2000 = (df['Year'] >= 0) & (df['Year'] < 30)

print('1900년대 2자리수 연도 데이터 수:', len(df[since1900]))
print('2000년대 2자리수 연도 데이터 수:', len(df[since2000]))
print('년도 빈값:', df['Year'].isna().sum())


1900년대 2자리수 연도 데이터 수: 0
2000년대 2자리수 연도 데이터 수: 0
년도 빈값: 0


In [6]:
df.sort_values('Year', ascending=True, inplace=True)

In [7]:
#퍼블리셔는 중요한 정보가 아니기 때문에 dropna에 걸리지 않게 특정값을 체우고 dropna를 실행
print(df['Publisher'].isna().sum())
df['Publisher'].fillna('--', inplace=True)
print(df['Publisher'].isna().sum())
df.dropna(inplace=True)

58
0


In [8]:
genres = df.Genre.value_counts()
genres.index

Index(['Action', 'Sports', 'Misc', 'Role-Playing', 'Shooter', 'Adventure',
       'Racing', 'Platform', 'Simulation', 'Fighting', 'Strategy', 'Puzzle'],
      dtype='object')

# 지역별 선호 게임 장르

In [101]:
def updateTitleLayout(f, title):
  f.update_layout(
    title=title,
    title_x=0.5,
    title_font_size = 25,
    font=dict(
        size=11,
        color="black"
    )
)

def getPreferenceGenre(area, start, end, top = 5):
  con = (df['Year'] >= start) & (df['Year'] <= end)
  data = df[con].groupby(['Genre'])[area].agg(**{'value':'sum'})
  data.sort_values(by='value', ascending=False, inplace=True)
  data.reset_index(inplace=True)
  return data[0:top]

def ShowPreferenceGenre(start, end):
  NA_Sales = getPreferenceGenre('NA_Sales', start, end)
  EU_Sales = getPreferenceGenre('EU_Sales', start, end)
  JP_Sales = getPreferenceGenre('JP_Sales', start, end)
  Other_Sales = getPreferenceGenre('Other_Sales', start, end)

  fig = make_subplots(rows = 2, cols = 2, subplot_titles=['NA','EU','JP','Other'], horizontal_spacing = 0.1, vertical_spacing=0.1)
  fig.add_bar (row = 1, col = 1, x = NA_Sales.Genre, y = NA_Sales.value)
  fig.add_bar (row = 1, col = 2, x = EU_Sales.Genre, y = EU_Sales.value)
  fig.add_bar (row = 2, col = 1, x = JP_Sales.Genre, y = JP_Sales.value)
  fig.add_bar (row = 2, col = 2, x = Other_Sales.Genre, y = Other_Sales.value)
  fig.update_layout(height = 800, width = 800, showlegend = False)
  fig.show()

ShowPreferenceGenre(1980, 2020)
ShowPreferenceGenre(1980, 1990)
ShowPreferenceGenre(1990, 2000)
ShowPreferenceGenre(2000, 2010)
ShowPreferenceGenre(2010, 2020)

# 연도별 게임 트렌드

In [10]:
def PreferenceByYear(area, target, start, end):
  con = (df['Year'] >= start) & (df['Year'] <= end)
  melt = pd.melt(df[con], id_vars=[target,'Year'], value_vars=area)
  melt_sum = melt.groupby(['Year', target])['value'].agg(**{'value':'sum'}).sort_values(by=['Year','value'], ascending=False)
  melt_sum.reset_index(inplace=True)
  
  fig = px.scatter(melt_sum, x = 'Year', y = 'value',
                color = 'Genre',
                hover_data=['Genre'],
                height=800, width=1200).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
  fig.update_layout(yaxis_title="출고량")
  for d in fig.data:
    d.update(mode='markers+lines')
  fig.show()

allArea = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales']
PreferenceByYear('Total_Sales', 'Genre', 1980, 2020)

# 출고량 분석

In [92]:
def getPreferenceGame(area, top = 5):
  melt = pd.melt(df, id_vars=['Name'], value_vars=area)
  melt_sum = melt.groupby(['Name'])['value'].agg(**{'value':'sum'}).sort_values(by=['value'], ascending=False)
  melt_sum.reset_index(inplace=True)
  return melt_sum.head(top)

All_PreferenceGame = getPreferenceGame('Total_Sales', 10)
fig = px.bar(All_PreferenceGame, x = 'Name', y = 'value',
       height=600, width=800)
fig.update_layout(xaxis_title="게임", yaxis_title="출고량")
updateTitleLayout(fig, '출고량 순위')
fig.show()

In [93]:
NA_PreferenceGame = getPreferenceGame('NA_Sales')
EU_PreferenceGame = getPreferenceGame('EU_Sales')
JP_PreferenceGame = getPreferenceGame('JP_Sales')
Other_PreferenceGame = getPreferenceGame('Other_Sales')

title = '지역별 출고량 순위'
fig = make_subplots(rows = 2, cols = 2, subplot_titles=['NA','EU','JP','Other'], horizontal_spacing = 0.1, vertical_spacing=0.2)
fig.add_bar (row = 1, col = 1, x = NA_PreferenceGame.Name, y = NA_PreferenceGame.value)
fig.add_bar (row = 1, col = 2, x = EU_PreferenceGame.Name, y = EU_PreferenceGame.value)
fig.add_bar (row = 2, col = 1, x = JP_PreferenceGame.Name, y = JP_PreferenceGame.value)
fig.add_bar (row = 2, col = 2, x = Other_PreferenceGame.Name, y = Other_PreferenceGame.value)
fig.update_layout(height = 1000, width = 1000, showlegend = False, title=title)
updateTitleLayout(fig, title)
fig.show()

In [13]:
def getPreferenceGameByYear(area, start, end):
  con = (df_melt['Year'] >= start) & (df_melt['Year'] <= end) & (df_melt['variable'] == area)
  data = df_melt[con]
  data = data.groupby(['Year','Name','Genre']).sum()  #플랫폼 다른것 합침
  data.reset_index(inplace=True)
  return data

In [14]:
def showPreferenceGameByYear(area, title, start, end, top = 5):
  data = getPreferenceGameByYear(area, start, end)
  data = data.groupby(['Year']).apply(lambda grb: grb.nlargest(top, 'value'))
  data.reset_index(level=0, drop=True, inplace=True)
  data.reset_index(drop=True, inplace=True)

  fig = px.bar(data, x = 'Year', y = 'value',
                color = 'Genre',
                text = 'Name',
                height=800, width=1200, title=title).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
  fig.update_layout(yaxis_title="출고량")
  updateTitleLayout(fig, title)
  fig.show()

df_melt = pd.melt(df, id_vars=['Name','Year','Genre','Platform'], value_vars=['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales'])
showPreferenceGameByYear('Total_Sales', '게임 순위 TOP 5', 1980, 2020)

In [15]:
showPreferenceGameByYear('NA_Sales', 'NA 게임 순위 TOP 5', 2010, 2016)
showPreferenceGameByYear('EU_Sales', 'EU 게임 순위 TOP 5', 2010, 2016)
showPreferenceGameByYear('JP_Sales', 'JP 게임 순위 TOP 5', 2010, 2016)
showPreferenceGameByYear('Other_Sales', 'Other 게임 순위 TOP 5', 2010, 2016)

# 가설 검정

In [ ]:
!pip install scipy --upgrade

In [49]:
def contingency(data1, data2):
  ct = pd.crosstab(data1, data2)
  chi2, pvalue, dof, data = chi2_contingency(ct, correction = False)
  print('chi2:',chi2, ' pvalue:', round(pvalue, 3), 'dof:', dof)
  return round(pvalue,3)

In [52]:
dfContingency = pd.DataFrame(columns=['Type', 'P-Value'],
                             data=[
                                   ['Name', contingency(df.Name, df.Total_Sales)],
                                   ['Year', contingency(df.Year, df.Total_Sales)],
                                   ['Genre', contingency(df.Genre, df.Total_Sales)],
                                   ['Platform', contingency(df.Platform, df.Total_Sales)],
                                   ['Publisher', contingency(df.Publisher, df.Total_Sales)]
                                   ])

px.bar(dfContingency, x = 'Type', y = 'P-Value', height=400, width=600)
#stats.ttest_ind(data1, data2)

chi2: 10412037.832327675  pvalue: 1.0 dof: 11530772
chi2: 75172.2656082835  pvalue: 0.0 dof: 38228
chi2: 11741.038845798827  pvalue: 0.0 dof: 11066
chi2: 39206.240973313994  pvalue: 0.0 dof: 30180
chi2: 209455.40143188208  pvalue: 1.0 dof: 580462


In [100]:
data = getPreferenceGameByYear('Total_Sales', 2010, 2020)
data.sort_values('value', ascending=False, inplace=True)
print(data.head(10))
data = data.groupby(['Genre']).apply(lambda grb: grb.nlargest(8, 'value'))
px.scatter(data, x = data.Year, y = data.value, color=data.Genre, width = 1000, height = 800).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))

      Year                            Name         Genre  value
2246  2013              Grand Theft Auto V        Action  37.77
982   2011  Call of Duty: Modern Warfare 3       Shooter  30.84
1728  2012      Call of Duty: Black Ops II       Shooter  29.73
90    2010         Call of Duty: Black Ops       Shooter  29.41
2163  2013            Call of Duty: Ghosts       Shooter  27.37
2820  2015       Call of Duty: Black Ops 3       Shooter  25.30
2495  2014  Call of Duty: Advanced Warfare       Shooter  21.89
391   2010              Kinect Adventures!          Misc  21.82
1550  2011     The Elder Scrolls V: Skyrim  Role-Playing  19.28
2548  2014                         FIFA 15        Sports  19.02


# 다음 분기에 어떤 게임을 설계해야 할까?

2010년도 부터 액션 슈팅 게임의 인기가 가장 높기 때문에
GTA나 콜오브 듀티같은 액션, 슈팅 게임을 만들어야 한다.